#Motor AI Practical Exercise **2**

> This exercise includes writing a Python script that combines 3 vector data (road bounds, road markings, buildings) into a single raster RGB mask with following pixel values:



*  road bound: [0, 0, 200]
*  buildings: [255, 0, 0]
*  road markings
  *   broken line: [0, 20, 10]
  *   cycle lane: [0, 40, 0]
  *   dashed line: [0, 45, 70]
  *   pedestrian crossing: [0, 100, 0]
  *   solid line: [0, 45, 0]
  *   stop line: [0, 85, 0]










In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 23.4 MB/s eta 0:00:00


In [1]:
#Import necessary libraries
import rasterio
import geopandas as gpd
import numpy as np
from rasterio import features

In [2]:
#Read raster data
with rasterio.open('/content/dop20rgb_386_5826_2022_grid_111.tif') as src:
    raster_meta = src.meta.copy()
    raster_shape = src.shape
    raster_crs = src.crs

In [3]:
#Read equivalent digitized vector data including road bound, road markings and buildings
road_bound = gpd.read_file('/content//dop20rgb_386_5826_2022_bounds_grid-111.shp')
buildings = gpd.read_file('/content//dop20rgb_386_5826_2022_buildings_grid-111.shp')
road_markings = gpd.read_file('/content//dop20rgb_386_5826_2022_markings_grid-111.shp')

In [5]:
#Reproject vector data to match the raster CRS
road_bound = road_bound.to_crs(raster_crs)
buildings = buildings.to_crs(raster_crs)
road_markings = road_markings.to_crs(raster_crs)

In [6]:
#Define pixel values for each feature
feature_colors = {
    'road_bound': (0, 0, 200),
    'buildings': (255, 0, 0),
    'broken_line': (0, 20, 10),
    'cycle_lane': (0, 40, 0),
    'dashed_line': (0, 45, 70),
    'pedestrian_crossing': (0, 100, 0),
    'solid_line': (0, 45, 0),
    'stop_line': (0, 85, 0)
}

In [7]:
#Create empty arrays with numpy for RGB color channels
red = np.zeros(raster_shape, dtype=np.uint8)
green = np.zeros(raster_shape, dtype=np.uint8)
blue = np.zeros(raster_shape, dtype=np.uint8)

In [8]:
#Define a rasterize function to rasterize features of vector layers
def rasterize_feature(gdf, color, buffer_size=0):
    if buffer_size > 0:
        gdf = gdf.copy()
        gdf['geometry'] = gdf['geometry'].buffer(buffer_size)
    shapes = ((geom, 1) for geom in gdf.geometry)
    mask = features.rasterize(shapes, out_shape=raster_shape, transform=raster_meta['transform'], fill=0, dtype=np.uint8)
    return mask * color

In [9]:
#Rasterize road bound
# Apply buffer to prevent (0,0,0) pixel values for road bounds
road_bound_mask = rasterize_feature(road_bound, 1, buffer_size=1)
red += road_bound_mask * feature_colors['road_bound'][0]
green += road_bound_mask * feature_colors['road_bound'][1]
blue += road_bound_mask * feature_colors['road_bound'][2]

#Rasterize buildings
buildings_mask = rasterize_feature(buildings, 1)
red += buildings_mask * feature_colors['buildings'][0]
green += buildings_mask * feature_colors['buildings'][1]
blue += buildings_mask * feature_colors['buildings'][2]

In [10]:
# Rasterize road markings
for feature, color in feature_colors.items():
    if feature != 'road_bound' and feature != 'buildings':
        feature_gdf = road_markings[road_markings['attributes'] == feature]
        feature_mask = rasterize_feature(feature_gdf, 1)
        red += feature_mask * color[0]
        green += feature_mask * color[1]
        blue += feature_mask * color[2]

In [11]:
#Stack color channels into a single RGB raster
rgb_raster = np.dstack((red, green, blue))

#Update metadata for the output raster
raster_meta.update(dtype=np.uint8, count=3)

#Save the final raster mask image
with rasterio.open('/content/Mask_image.tif', 'w', **raster_meta) as dst:
    dst.write(rgb_raster.transpose(2, 0, 1))

print("Raster mask is created successfully!")

Raster mask is created successfully!
